<a href="https://colab.research.google.com/github/aloysiusw/redditsearchengine/blob/main/RedditSearchEngine_InformationRetrieval_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initial setup

In [37]:
#import dependencies
import glob
import os
import json
import pandas as pd
import numpy as np

import nltk
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import stopwords
nltk.download('stopwords')

from datetime import datetime
import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#mount drive, no longer used for public demo
#from google.colab import drive 
#drive.mount('/content/drive')

In [4]:
!wget 'https://github.com/aloysiusw/redditsearchengine/raw/main/ir-rawjson-reddit.zip'

--2023-01-11 15:55:18--  https://github.com/aloysiusw/redditsearchengine/raw/main/ir-rawjson-reddit.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aloysiusw/redditsearchengine/main/ir-rawjson-reddit.zip [following]
--2023-01-11 15:55:18--  https://raw.githubusercontent.com/aloysiusw/redditsearchengine/main/ir-rawjson-reddit.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17018837 (16M) [application/zip]
Saving to: ‘ir-rawjson-reddit.zip’

ir-rawjson-reddit.z 100%[===================>]  16.23M  --.-KB/s    in 0.08s   

2023-01-11 15:55:19 (192 MB/s) - ‘ir-rawjson-reddit.zip’ saved [17018

In [5]:
!unzip '/content/ir-rawjson-reddit.zip'

Archive:  /content/ir-rawjson-reddit.zip
   creating: json raw/
  inflating: json raw/---ShineyHiney---_Recommendation request ladys steel toed boots. Help!_sy6jxx.json  
  inflating: json raw/--Athena--_1950_s Fridgidaire stove by General Motors. In working order._912dfa.json  
  inflating: json raw/-Altra-_Soviet Surplus and Asbestos_tauq2s.json  
  inflating: json raw/-SpaghettiCat-_Seeking Tips for Cleaning  Maintenance of Clarks Leather Boots_svzvs2.json  
  inflating: json raw/0621FiST_Truly bought for life quackenbush 22 youth rifle._t6t6bh.json  
  inflating: json raw/0Bradda_Car Sun Shade Request_u5h4ja.json  
  inflating: json raw/13mx_Easter Basket to use every Easter_tj06vw.json  
  inflating: json raw/19Chris96_This freaking Steelcase chair I found today at a Goodwill for $1. ONE DOLLAR!!! It TRUMPS my Herman Miller Equa find._jt01ar.json  
  inflating: json raw/1point21_$50 Weber on Craigslist! Great shape too, will probably just replace the grates and flavorizers_cuxnoy.

#Converting the raw data to a more useable format

In [11]:
#list all files in directory
fileLocPath = '/content/json raw'
!ls fileLocPath

ls: cannot access 'fileLocPath': No such file or directory


In [12]:
#define function to grab the files
def get_files(filepath):
	all_files = []
	for root, dirs, files in os.walk(filepath):
		files = glob.glob(os.path.join(root,'*.json')) #use json files
		for f in files :
			all_files.append(os.path.abspath(f))
			
	return all_files

In [13]:
#get files from drive
content = get_files(fileLocPath)

In [14]:
#create array
redditContent = []

#for all files, add to dataframe
for data in content:
	with open(data) as doc:
		exp = json.load(doc)
		redditContent.append(exp)

In [15]:
#test display
redditContent[1]

{'title': 'Suitcases. Infrequent travellers. UK.',
 'name': 't3_u4znrw',
 'url': 'https://www.reddit.com/r/BuyItForLife/comments/u4znrw/suitcases_infrequent_travellers_uk/',
 'selftext': "I have done my best to research on here as I appreciate there are many topics on luggage/suitcases already.\n\n There's a very American bent to most of the discussion though so I thought it was worth asking for UK-focussed recommendations if anyone has them.\n\n We also only travel once or twice a year, so I wonder if something like Briggs and Riley are a bit overkill for our purposes (don't particularly want to spend that much).\n\nI'm after 2 checked luggage bags/suitcases. Need to hold enough stuff for a week or so. Price-wise I guess about £250 per suitcase would be what I'd be aiming to stay under (but if there's a compelling case to go a bit over I'm happy to hear it).\n\nWe did get given a load of John Lewis vouchers when we got married recently so buying from there would be *ideal* but not a r

In [16]:
#convert to pandas dataframe  
redditDF = pd.DataFrame(redditContent)

In [17]:
#show dataframe dimension
redditDF.shape

(1562, 18)

In [18]:
#show top 3 data 
redditDF.head(3)

,title,name,url,selftext,score,upvote_ratio,permalink,id,author,link_flair_text,num_comments,over_18,spoiler,pinned,locked,distinguished,created_utc,comments
0,Baracuta G9 Harrington Jacket - Any thoughts?,t3_uatx76,https://www.reddit.com/r/BuyItForLife/comments...,Does anyone have this jacket? Is this somethin...,6,1.00,/r/BuyItForLife/comments/uatx76/baracuta_g9_ha...,uatx76,anonymouspsy,Discussion,4,False,False,False,False,None,1.650804e+09,"[{'author': 'AutoModerator', 'id': 'i5zsjhm', ..."
1,Suitcases. Infrequent travellers. UK.,t3_u4znrw,https://www.reddit.com/r/BuyItForLife/comments...,I have done my best to research on here as I a...,12,0.73,/r/BuyItForLife/comments/u4znrw/suitcases_infr...,u4znrw,Migraine-,[Request],16,False,False,False,False,None,1.650121e+09,"[{'author': 'bladepen', 'id': 'i4z2loj', 'scor..."
2,1980s (??) Surplus Canadian Army Parade Boots,t3_uabzha,https://www.reddit.com/gallery/uabzha,,93,0.97,/r/BuyItForLife/comments/uabzha/1980s_surplus_...,uabzha,zombienudist,Vintage,15,False,False,False,False,None,1.650739e+09,"[{'author': 'zombienudist', 'id': 'i5wqymt', '..."


In [19]:
#show bottom 3 data
redditDF.tail(3)

,title,name,url,selftext,score,upvote_ratio,permalink,id,author,link_flair_text,num_comments,over_18,spoiler,pinned,locked,distinguished,created_utc,comments
1559,My only razor since I started shaving - Gillet...,t3_gjn023,https://i.redd.it/94oqmomjmqy41.jpg,,2216,0.97,/r/BuyItForLife/comments/gjn023/my_only_razor_...,gjn023,telemakos64,Bathroom,148,False,False,False,False,None,1.589465e+09,"[{'author': 'hermeez', 'id': 'fqltczm', 'score..."
1560,baking steel for frozen pizza,t3_t7c2c9,https://www.reddit.com/r/BuyItForLife/comments...,"Looking for a pizza pan, seen baking steels hi...",0,0.50,/r/BuyItForLife/comments/t7c2c9/baking_steel_f...,t7c2c9,humblesunset,[Request],8,False,False,False,False,None,1.646495e+09,"[{'author': 'ortolon', 'id': 'hzgt7fq', 'score..."
1561,BIFL monogrammed beer steins to replace flimsy...,t3_tcsslf,https://www.reddit.com/r/BuyItForLife/comments...,I purchased monogrammed beer steins for a groo...,4,0.83,/r/BuyItForLife/comments/tcsslf/bifl_monogramm...,tcsslf,ZigguratOfUr,[Request],1,False,False,False,False,None,1.647125e+09,"[{'author': 'tak0kat0', 'id': 'i0fdvjf', 'scor..."


In [21]:
#convert the dataframe to a csv
redditDF.to_csv('redditDataset-original.csv', encoding='utf-8', index=False)

#Cleaning the data

In [22]:
redditDF = pd.read_csv('/content/redditDataset-original.csv', lineterminator='\n')
redditDF = redditDF.replace(np.nan, '', regex=True)

In [23]:
#remove redundant and unused columns
redditDF = redditDF.drop(columns=['name','over_18','spoiler','pinned','locked','distinguished','comments'])

In [24]:
redditDF.head(3)

,title,url,selftext,score,upvote_ratio,permalink,id,author,link_flair_text,num_comments,created_utc
0,Baracuta G9 Harrington Jacket - Any thoughts?,https://www.reddit.com/r/BuyItForLife/comments...,Does anyone have this jacket? Is this somethin...,6,1.00,/r/BuyItForLife/comments/uatx76/baracuta_g9_ha...,uatx76,anonymouspsy,Discussion,4,1.650804e+09
1,Suitcases. Infrequent travellers. UK.,https://www.reddit.com/r/BuyItForLife/comments...,I have done my best to research on here as I a...,12,0.73,/r/BuyItForLife/comments/u4znrw/suitcases_infr...,u4znrw,Migraine-,[Request],16,1.650121e+09
2,1980s (??) Surplus Canadian Army Parade Boots,https://www.reddit.com/gallery/uabzha,,93,0.97,/r/BuyItForLife/comments/uabzha/1980s_surplus_...,uabzha,zombienudist,Vintage,15,1.650739e+09


In [25]:
redditDF.head(3)

,title,url,selftext,score,upvote_ratio,permalink,id,author,link_flair_text,num_comments,created_utc
0,Baracuta G9 Harrington Jacket - Any thoughts?,https://www.reddit.com/r/BuyItForLife/comments...,Does anyone have this jacket? Is this somethin...,6,1.00,/r/BuyItForLife/comments/uatx76/baracuta_g9_ha...,uatx76,anonymouspsy,Discussion,4,1.650804e+09
1,Suitcases. Infrequent travellers. UK.,https://www.reddit.com/r/BuyItForLife/comments...,I have done my best to research on here as I a...,12,0.73,/r/BuyItForLife/comments/u4znrw/suitcases_infr...,u4znrw,Migraine-,[Request],16,1.650121e+09
2,1980s (??) Surplus Canadian Army Parade Boots,https://www.reddit.com/gallery/uabzha,,93,0.97,/r/BuyItForLife/comments/uabzha/1980s_surplus_...,uabzha,zombienudist,Vintage,15,1.650739e+09


In [26]:
#Convert the time column into a readable format
redditDateAdjustedDF = redditDF.copy()
redditDateAdjustedDF['timestamp_utc'] = pd.to_datetime(redditDateAdjustedDF['created_utc'], unit='s', utc=True)
redditDateAdjustedDF = redditDateAdjustedDF.sort_values(by='created_utc', ascending=False)

In [27]:
redditDateAdjustedDF.head(3)

,title,url,selftext,score,upvote_ratio,permalink,id,author,link_flair_text,num_comments,created_utc,timestamp_utc
1404,BIFL Monthly Steals & Deals Thread 2022-05,https://www.reddit.com/r/BuyItForLife/comments...,Found an amazing deal? Share them here so othe...,3,1.0,/r/BuyItForLife/comments/ug28k2/bifl_monthly_s...,ug28k2,Robot_ninja_pirate,Offical Discussion,0,1.651419e+09,2022-05-01 15:26:31+00:00
609,Saybrook Pillow VS Coop Eden Pillow,https://www.reddit.com/r/BuyItForLife/comments...,Hello\n\nI was looking into the reviews of man...,2,1.0,/r/BuyItForLife/comments/ug20wg/saybrook_pillo...,ug20wg,Thamer_com,Discussion,3,1.651418e+09,2022-05-01 15:16:06+00:00
1111,Sheets that won’t attract animal hair?,https://www.reddit.com/r/BuyItForLife/comments...,So I currently have two sets of sheets from Br...,1,0.6,/r/BuyItForLife/comments/ufsxms/sheets_that_wo...,ufsxms,spooky_kid96,[Request],3,1.651383e+09,2022-05-01 05:30:39+00:00


In [28]:
#Get the unique flairs of the dataset
redditDateAdjustedDF.link_flair_text.unique()

array(['Offical Discussion', 'Discussion', '[Request]', 'Repair',
       '[Request] Answered!', 'Currently sold', 'Vintage', 'Review',
       'BIFL Skills', 'Meta', 'Warranty', '', "Men's Clothing", 'Kitchen',
       'Appliances', 'Office Supplies', 'Travel and Outdoors',
       'Automotive', 'Toys and Hobbies', 'Jewelry or Accessories',
       'Electronics', 'Household', 'Tools & home Improvement',
       'Bags and Luggage', 'Bathroom', 'Shoes', 'Other', 'Furniture',
       'Clothing', 'Possibly Incorrect', 'Grill', 'Travel & Outdoors',
       'Bags & Luggage', 'Work truck', '*Zojirushi',
       'Tools &amp; home Improvement', 'Toys & Hobbies', 'Mod approved',
       '/R/ALL', 'Jewelry & Accessories', 'Show & Tell',
       'r/all Show & Tell', 'SHITPOST', 'Outdoors', 'tools'], dtype=object)

In [29]:
#Drop entries with the irrelevant flairs
redditFlairAdjustedDF = redditDateAdjustedDF.copy()
redditFlairAdjustedDF.drop(redditFlairAdjustedDF.index[redditFlairAdjustedDF['link_flair_text'] == 'SHITPOST'], inplace=True);
redditFlairAdjustedDF.drop(redditFlairAdjustedDF.index[redditFlairAdjustedDF['link_flair_text'] == 'Meta'], inplace=True);
redditFlairAdjustedDF.drop(redditFlairAdjustedDF.index[redditFlairAdjustedDF['link_flair_text'] == 'Offical Discussion'], inplace=True);

In [30]:
#Check that the flairs are not there anymore
redditFlairAdjustedDF.link_flair_text.unique()

array(['Discussion', '[Request]', 'Repair', '[Request] Answered!',
       'Currently sold', 'Vintage', 'Review', 'BIFL Skills', 'Warranty',
       '', "Men's Clothing", 'Kitchen', 'Appliances', 'Office Supplies',
       'Travel and Outdoors', 'Automotive', 'Toys and Hobbies',
       'Jewelry or Accessories', 'Electronics', 'Household',
       'Tools & home Improvement', 'Bags and Luggage', 'Bathroom',
       'Shoes', 'Other', 'Furniture', 'Clothing', 'Possibly Incorrect',
       'Grill', 'Travel & Outdoors', 'Bags & Luggage', 'Work truck',
       '*Zojirushi', 'Tools &amp; home Improvement', 'Toys & Hobbies',
       'Mod approved', '/R/ALL', 'Jewelry & Accessories', 'Show & Tell',
       'r/all Show & Tell', 'Outdoors', 'tools'], dtype=object)

In [31]:
#Check the new dimension
redditFlairAdjustedDF.shape

(1543, 12)

In [32]:
redditFlairAdjustedDF.to_csv('/content/redditDataset-cleaned.csv', encoding='utf-8', index=False)

#Pre-processing the data - language processing


In [33]:
redditDF = pd.read_csv('/content/redditDataset-cleaned.csv', lineterminator='\n')
redditDF = redditDF.replace(np.nan, '', regex=True)

In [34]:
#Create keyword column with the combined title + content
redditDF['keywords'] = redditDF['title'] + redditDF['selftext']
redditDF['keywords'] = redditDF['keywords'].astype(str)

In [35]:
#Sets all the content in each entry to be lowercase
redditDF['keywords'] = [entry.lower() for entry in redditDF['keywords']]

In [38]:
#Lemmatize the entries
redditDF['keywords'] = [nltk.WordNetLemmatizer().lemmatize(entry) for entry in redditDF.keywords] 

In [39]:
#Tokenize the keywords (separating each word into its own entry)
redditDF['keywords'] = [word_tokenize(entry) for entry in redditDF.keywords] 
#redditDF['keywords'] = redditDF['keywords'].astype(str)

In [40]:
#Remove stop words
stop_words = set(stopwords.words('english'))
indCounter = 0
for entry in redditDF.keywords:
  filteredSentence = [w for w in entry if not w.lower() in stop_words]
  for w in entry:
    if w not in stop_words:
      w=w.replace("\{.",'')
      w=w.replace("'",'')
      w=w.replace(" ",'')
      w=w.replace("\].",'')
      filteredSentence.append(w)

  redditDF.at[indCounter,'keywords'] = filteredSentence
  indCounter += 1

In [41]:
#Convert to set remove duplicates
redditDF['keywords'] = [list(set(entry)) for entry in redditDF.keywords] 

In [42]:
indCounter = 0
for entry in redditDF.keywords:
  newValue=",".join(entry)
  redditDF.at[indCounter,'keywords'] = newValue
  indCounter += 1

In [43]:
print(redditDF.loc[0,'keywords'])

reviews,shipping,s,https,many,pillows,'s,//www.amazon.com/saybrook-premium-adjustable-loft-pillow/dp/b08dp626vn,incredibly,30,two,problem,know,looking,want,case,tried,hand,fixed,price,),.,excluded,better,conclusion,?,average,,,lot,[,:,pillow,eden,zip,complaining,//www.amazon.com/coop-home-goods-adjustable-hypoallergenic/dp/b01lyu7v4s,older,came,included,old,],vs,filling,saybrook,coop,pillowhello,$,less,anyone,seems,(,think


In [44]:
redditDF.to_csv('/content/redditDataset-languageprocessed.csv', encoding='utf-8', index=False)

#Pre-processing the data - decay rating

In [45]:
redditDF = pd.read_csv('/content/redditDataset-languageprocessed.csv', lineterminator='\n')
redditDF = redditDF.replace(np.nan, '', regex=True)

In [46]:
#Establish today's date
todayTime = datetime.now()

In [47]:
#Constant for decay factor formula
alpha=1/250000000000

In [48]:
#Calculate the decay factor for each entry
timeDifference = todayTime.timestamp()-redditDF['created_utc']
decay=np.exp(-alpha*timeDifference)

In [49]:
#Calculate decayed rating and insert to database
redditDF['score_decayed']=redditDF['score']*decay

In [50]:
redditDF.head()

,title,url,selftext,score,upvote_ratio,permalink,id,author,link_flair_text,num_comments,created_utc,timestamp_utc,keywords,score_decayed
0,Saybrook Pillow VS Coop Eden Pillow,https://www.reddit.com/r/BuyItForLife/comments...,Hello\n\nI was looking into the reviews of man...,2,1.00,/r/BuyItForLife/comments/ug20wg/saybrook_pillo...,ug20wg,Thamer_com,Discussion,3,1.651418e+09,2022-05-01 15:16:06+00:00,"reviews,shipping,s,https,many,pillows,'s,//www...",1.999824
1,Sheets that won’t attract animal hair?,https://www.reddit.com/r/BuyItForLife/comments...,So I currently have two sets of sheets from Br...,1,0.60,/r/BuyItForLife/comments/ufsxms/sheets_that_wo...,ufsxms,spooky_kid96,[Request],3,1.651383e+09,2022-05-01 05:30:39+00:00,"like,starting,two,problem,years,looking,want,a...",0.999912
2,Rejection of servicing/repair in this sub?,https://www.reddit.com/r/BuyItForLife/comments...,Is there a reason why most posts I've observed...,226,0.95,/r/BuyItForLife/comments/ufs45f/rejection_of_s...,ufs45f,jessecakeindustries,Repair,58,1.651380e+09,2022-05-01 04:35:44+00:00,"s,service/repair,built,servicing/repair,made,l...",225.980047
3,Briggs and Riley - Baseline vs ZDX,https://www.reddit.com/r/BuyItForLife/comments...,I'm an on the road forever nomad who lives out...,2,1.00,/r/BuyItForLife/comments/ufq3y2/briggs_and_ril...,ufq3y2,ElvisGrizzly,Discussion,2,1.651372e+09,2022-05-01 02:32:14+00:00,"s,avoiding,looks,like,expensive,'s,riley,two,r...",1.999823
4,Breathable storage bag for sweaters,https://www.reddit.com/r/BuyItForLife/comments...,I am looking for recommendations on breathable...,2,1.00,/r/BuyItForLife/comments/ufokkt/breathable_sto...,ufokkt,LuckOTheeIrish,[Request],3,1.651367e+09,2022-05-01 01:03:07+00:00,"reviews,bag,ll,storing,term,look,show,plastic,...",1.999823


In [51]:
redditDF[['created_utc','timestamp_utc','score','score_decayed']]

,created_utc,timestamp_utc,score,score_decayed
0,1.651418e+09,2022-05-01 15:16:06+00:00,2,1.999824
1,1.651383e+09,2022-05-01 05:30:39+00:00,1,0.999912
2,1.651380e+09,2022-05-01 04:35:44+00:00,226,225.980047
3,1.651372e+09,2022-05-01 02:32:14+00:00,2,1.999823
4,1.651367e+09,2022-05-01 01:03:07+00:00,2,1.999823
...,...,...,...,...
1538,1.463109e+09,2016-05-13 03:05:40+00:00,2445,2442.943701
1539,1.445287e+09,2015-10-19 20:42:53+00:00,4459,4454.932299
1540,1.427129e+09,2015-03-23 16:38:53+00:00,2318,2315.717209
1541,1.418744e+09,2014-12-16 15:27:30+00:00,2178,2175.782104


In [52]:
redditDF.to_csv('redditDataset-final.csv', encoding='utf-8', index=False)

#The search engine itself

In [53]:
redditDF = pd.read_csv('/content/redditDataset-final.csv', lineterminator='\n')
redditDF = redditDF.replace(np.nan, '', regex=True)

In [54]:
redditDF.head(3)

,title,url,selftext,score,upvote_ratio,permalink,id,author,link_flair_text,num_comments,created_utc,timestamp_utc,keywords,score_decayed
0,Saybrook Pillow VS Coop Eden Pillow,https://www.reddit.com/r/BuyItForLife/comments...,Hello\n\nI was looking into the reviews of man...,2,1.00,/r/BuyItForLife/comments/ug20wg/saybrook_pillo...,ug20wg,Thamer_com,Discussion,3,1.651418e+09,2022-05-01 15:16:06+00:00,"reviews,shipping,s,https,many,pillows,'s,//www...",1.999824
1,Sheets that won’t attract animal hair?,https://www.reddit.com/r/BuyItForLife/comments...,So I currently have two sets of sheets from Br...,1,0.60,/r/BuyItForLife/comments/ufsxms/sheets_that_wo...,ufsxms,spooky_kid96,[Request],3,1.651383e+09,2022-05-01 05:30:39+00:00,"like,starting,two,problem,years,looking,want,a...",0.999912
2,Rejection of servicing/repair in this sub?,https://www.reddit.com/r/BuyItForLife/comments...,Is there a reason why most posts I've observed...,226,0.95,/r/BuyItForLife/comments/ufs45f/rejection_of_s...,ufs45f,jessecakeindustries,Repair,58,1.651380e+09,2022-05-01 04:35:44+00:00,"s,service/repair,built,servicing/repair,made,l...",225.980047


In [56]:
try:
  running = True
  while(running):
    #Variables to track the sorting arguments
    sortingMethod = "Default (top)"
    useDecayedRating = False
    useUnsortedList = False
    sortByRecent = False
    sortByOldest = False
    sortByLowest = False
    outputLimit = 10

    #Accept user input
    print("Input -help for guide.\n")
    termInput = input('> ')
    #Set input to lowercase
    parsedInput = termInput.lower()

    #Help argument
    if(parsedInput == '-help'):
      print("Input your query in the the search box.\nThere are several search parameters to choose from,\n")
      print("'-decay' or '-d' : Sort from the highest score, adjusted by decay factor\n'-recent' or '-r' : Sort by the newest data\n'-oldest' or '-o' : Sort by the oldest data\n'-lowest' or '-l' : Sort by the lowest scoring data\n'-*' : Replace * with a numeric value and only show that amount of results\nDefault sorting method is the top highest scoring submissions\n")
    #If input is not exit, continue with the program
    elif(parsedInput != 'exit'):
      #Parse the input, extracting the parameters separated by " -" into a separate list
      parsedInput = termInput.split(' -')
      #Get query length
      queryLength = len(parsedInput)
      #print('Query length:', queryLength)
      #If first query is True/not empty
      if(parsedInput[0]):
        queriedInput = parsedInput[0]
        print('Query:', queriedInput)
        if(queryLength > 1):
          ind=0
          #print(queryLength)
          #Parse through the input list and updates boolean based on arguments
          while(ind < queryLength):
            optionValue = parsedInput[ind]
            if(optionValue == 'decay' or optionValue == 'd'):
              useDecayedRating=True
              sortingMethod = "Top, with decay rating"
            elif(optionValue == 'unsorted' or optionValue == 'u'):
              useUnsortedList=True
              sortingMethod = "Unsorted"
            elif(optionValue == 'recent' or optionValue =='r'):
              sortByRecent=True
              sortingMethod = "Most recent"
            elif(optionValue == 'oldest' or optionValue == 'o'):
              sortByOldest=True
              sortingMethod = "Oldest"
            elif(optionValue == 'lowest' or optionValue == 'l'):
              sortByLowest=True
              sortingMethod = "Lowest"
            elif(optionValue.isnumeric()):
              outputLimit=optionValue
            ind+=1

        #Makes a dataframe based on the input
        searchResultDF = redditDF[redditDF['keywords'].str.contains(queriedInput)]

        #Check if there are any results
        print('Found results:', len(searchResultDF))
        if(len(searchResultDF) != 0):
          #If there are results, adjust according to the boolean
          if(useDecayedRating):
            searchResultDF = searchResultDF.sort_values('score_decayed', ascending=False)
          elif(useUnsortedList):
            searchResultDF
          elif(sortByRecent):
            searchResultDF = searchResultDF.sort_values('created_utc', ascending=False)
          elif(sortByOldest):
            searchResultDF = searchResultDF.sort_values('created_utc',ascending=True)
          elif(sortByLowest):
            searchResultDF = searchResultDF.sort_values('score', ascending=True)
          else:
            searchResultDF = searchResultDF.sort_values('score', ascending=False)
        else:
          print("No results found.")
        #print(searchResultDF.columns)
        i=0
        i=int(i)
        if(len(searchResultDF)>0):
          print("Displayed output:",outputLimit)
          print("Sorted by:",sortingMethod,"\n")
        #For results in the result dataframe
        for idx in searchResultDF.iterrows():
          while(i < int(outputLimit) and i < len(searchResultDF)):
            
            valueFlair = searchResultDF.iloc[i,8]
            if(valueFlair != ""):
              print("Flair:",valueFlair)

            valueTitle = searchResultDF.iloc[i,0]  
            print("Title:\n",valueTitle)

            valueTimestamp = searchResultDF.iloc[i,11]
            print("Submitted:",valueTimestamp)

            valueScore = searchResultDF.iloc[i,3]
            print("Rating:",valueScore)

            if(useDecayedRating):
              valueDecayed = searchResultDF.iloc[i,13]
              print("Rating with Decayed Factor:",valueDecayed)     

            valueRatio = searchResultDF.iloc[i,4]
            print("Upvote ratio:",valueRatio)

            valueNumComments = searchResultDF.iloc[i,9]
            print("No. of comments:",valueNumComments)

            valueURL = searchResultDF.iloc[i,1]
            print("Link:",valueURL)

            valueKeywords = searchResultDF.iloc[i,12]
            print("Keywords:",valueKeywords)

            valueBody = searchResultDF.iloc[i,2]
            if(valueBody != ""):
              print("Body:\n",valueBody)
            print("\n")
            i+=1
      else:
        print('No input found.')
    else:
      #Exit boolean
      running = False
except KeyboardInterrupt:
  pass
print("Successfully exited.")

Input -help for guide.

> -help
Input your query in the the search box.
There are several search parameters to choose from,

'-decay' or '-d' : Sort from the highest score, adjusted by decay factor
'-recent' or '-r' : Sort by the newest data
'-oldest' or '-o' : Sort by the oldest data
'-lowest' or '-l' : Sort by the lowest scoring data
'-*' : Replace * with a numeric value and only show that amount of results
Default sorting method is the top highest scoring submissions

Input -help for guide.

> book
Query: book
Found results: 14
Displayed output: 10
Sorted by: Default (top) 

Flair: Discussion
Title:
 Found this on Facebook today… thought some of you might enjoy it.
Submitted: 2022-03-20 18:19:46+00:00
Rating: 7360
Upvote ratio: 0.93
No. of comments: 391
Link: https://i.redd.it/y9p9q091zko81.jpg
Keywords: found,today…,enjoy,might,facebook,.,thought


Flair: Review
Title:
 Moleskine notebook - new vs 8 years old
Submitted: 2021-03-10 12:12:21+00:00
Rating: 2380
Upvote ratio: 0.95
No. 